In [1]:
# These magics make working in the notebook a little easier
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import paderbox as pb

In [3]:
# Import ASR Task - This is the only thing needed to start the whole training
from espnet2.tasks.asr import ASRTask

This is where default works


In [4]:
# This is a little bit hacky: ESPnet uses the argparser for everything (terrible idea in my opinion), 
# so we have to fake the command line arguments by directly writing the config into a `argparse.Namespace`.
# You can write the config by yourself or read in a config file of a finished training. It is in ".../exp/asr_train.../config.yaml"
import argparse

# TODO: insert path to config file
#config_file = '/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_train_asr_transformer_raw_char/config.yaml'
config_file = '/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_SpkVec/config.yaml'
config = pb.io.load(config_file)

args = argparse.Namespace(**config)
args

Namespace(accum_grad=8, allow_variable_data_keys=True, batch_bins=1000000, batch_size=6, batch_type='folded', best_model_criterion=[['valid', 'acc', 'max']], bpemodel=None, chunk_length=500, chunk_shift_ratio=0.5, cleaner=None, collect_stats=False, config='conf/train_asr_transformer.yaml', ctc_conf={'dropout_rate': 0.0, 'ctc_type': 'builtin', 'reduce': True, 'ignore_nan_grad': False}, cudnn_benchmark=False, cudnn_deterministic=True, cudnn_enabled=True, decoder='transformer', decoder_conf={'attention_heads': 4, 'linear_units': 2048, 'num_blocks': 6, 'dropout_rate': 0.1, 'positional_dropout_rate': 0.1, 'self_attention_dropout_rate': 0.0, 'src_attention_dropout_rate': 0.0}, dist_backend='nccl', dist_init_method='env://', dist_launcher=None, dist_master_addr=None, dist_master_port=None, dist_rank=None, dist_world_size=None, distributed=False, dry_run=False, early_stopping_criterion=['valid', 'loss', 'min'], encoder='transformer', encoder_conf={'output_size': 256, 'attention_heads': 4, 'lin

In [5]:
# Build the model
model = ASRTask.build_model(args)


In [6]:
# Another hacky thing: You have to be in the experiment directory for the ASRTask to find the data files, so change directory there
import os
# TODO: insert path to fearless directory in your home
os.chdir('/net/home/vivekkan/my_project/espnet/egs2/fearless/asr1')

In [15]:
# Build the iterator that loads the data. Or what ESPnet does: Build a factory that returns an iterator for that
from espnet2.train.distributed_utils import DistributedOption
from espnet2.utils.build_dataclass import build_dataclass

# Set mode='valid' for validation data
iter_factory = ASRTask.build_iter_factory(args, distributed_option=build_dataclass(DistributedOption, args), mode='train')
iterator = iter_factory.build_iter(8)

In [8]:
ASRTask.build_model

<bound method ASRTask.build_model of <class 'espnet2.tasks.asr.ASRTask'>>

In [16]:
# Get one batch from the iterator, in the format that ESPnet requires it
it = iter(iterator)
batch = next(it)

In [17]:
len(batch)

2

In [18]:
# Batch consists of the example IDs and the data in the format specified in the args (train_data_path_and_name_and_type config value(s))
# The keys in data should correspond to the arguments of ESPnetASRModel.forward
example_ids, data = batch
example_ids, data.keys()

(['FIDO1-FS02_ASR_track2_train_08524',
  'FD1-FS02_ASR_track2_train_34078',
  'FD1-FS02_ASR_track2_train_26235',
  'FD1-FS02_ASR_track2_train_24148',
  'FD1-FS02_ASR_track2_train_14923',
  'FAO2-FS02_ASR_track2_train_20762'],
 dict_keys(['speech', 'speech_lengths', 'text', 'text_lengths', 'speaker_id', 'speaker_id_lengths']))

In [20]:
data['speech'][3].shape

torch.Size([52800])

In [ ]:
# Pass one example through the model. The output countains the loss and some statistics to report to tensorboard.
# If you don't want to compute the backward step in the notebook, it is better to use torch.no_grad to save some memory and speed things up
with torch.no_grad():
    model_out = model(**data)
loss, stats, weight = model_out
loss, stats, weight

In [ ]:
# Pass one example through the model. The output countains the loss and some statistics to report to tensorboard.
# If you don't want to compute the backward step in the notebook, it is better to use torch.no_grad to save some memory and speed things up
with torch.no_grad():
    model_out = model(**data)
loss, stats, weight, featsU ,featsULen = model_out
loss, stats, weight

In [ ]:
data['speech'].shape


In [ ]:
speech = data['speech'].unsqueeze(0).to(getattr(torch, 'float32'))

In [ ]:
lengths = speech.new_full([1], dtype=torch.long, fill_value=speech.size(1))

In [ ]:
lengths

In [ ]:
featsU

In [ ]:
feats1[0][:][0]

In [ ]:
featsU.mean()

In [ ]:
featsU[2]

In [ ]:
data['speaker_id'].shape

In [ ]:
testX= torch.Tensor
for i in range(len(featsU)):
    for j in range(len(featsU[i])):
        testX[i,j,:]= torch.cat((featsU[i][j],data['speaker_id'][i]), dim=0)

In [ ]:
data['speaker_id'][0].reshape(3,83)

In [ ]:
featsU[0][0].shape

In [ ]:
len(featsU[0])

In [ ]:
data['speaker_id'].shape

In [ ]:
testSpeak=tf.expand_dims(data['speaker_id'],1)

In [ ]:
import tensorflow as tf

In [ ]:

#Important
K=featsU.shape[1]

In [ ]:
#Important
SpkFeat= data['speaker_id'].unsqueeze(1).repeat(1, K, 1)

In [ ]:
SpkFeat.shape

In [ ]:
#Important
SpkFeat=torch.cat((SpkFeat,featsU),dim=2)

In [ ]:
SpkFeat.shape

In [ ]:
feats_lengths

In [ ]:
data['speaker_id_lengths']

In [ ]:
featsU.mean()

In [ ]:
TestUn.mean()